In [274]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

In [275]:
farmgate = pd.read_csv('../data/raw/FarmgatePrice.csv')
retail = pd.read_csv('../data/raw/RetailPrice.csv')
wholesale = pd.read_csv('../data/raw/WholesalePrice.csv')
production = pd.read_csv('../data/raw/VolumeOfProductionByMetricTons.csv')
harvested = pd.read_csv('../data/raw/AreaHarvestedInHectares.csv')
cpi = pd.read_csv('../data/raw/ConsumerPriceIndex.csv')

def melt_df(df, column_1, column_2):
    df = df.copy()
    
    df_long = df.melt(id_vars=[column_1, column_2], var_name='Date', value_name='Price')
    
    return df_long

farmgate_copy = melt_df(farmgate, "Geolocation", "Commodity")
retail_copy = melt_df(retail, "Geolocation", "Commodity")
wholesale_copy = melt_df(wholesale, "Geolocation", "Commodity")
production_copy = melt_df(production, "Ecosystem/Croptype", "Geolocation")
harvested_copy = melt_df(harvested, "Ecosystem/Croptype", "Geolocation")
cpi_copy = melt_df(cpi, "Geolocation", "Commodity Description")

In [276]:
def column_name_cleaning(df, column_name):
    df = df.copy()
    
    df[column_name] = df[column_name].str.replace(r'^\.\.', '', regex=True).str.strip()
    df[column_name] = df[column_name].str.lower().str.strip()
    
    return df

farmgate = column_name_cleaning(farmgate, 'Geolocation')
retail = column_name_cleaning(retail, 'Geolocation')
wholesale = column_name_cleaning(wholesale, 'Geolocation')
production = column_name_cleaning(production, 'Ecosystem/Croptype')
harvested = column_name_cleaning(harvested, 'Ecosystem/Croptype')
cpi = column_name_cleaning(cpi, 'Geolocation')

farmgate = column_name_cleaning(farmgate, 'Commodity')
retail = column_name_cleaning(retail, 'Commodity')
wholesale = column_name_cleaning(wholesale, 'Commodity')
production = column_name_cleaning(production, 'Geolocation')
harvested = column_name_cleaning(harvested, 'Geolocation')
cpi = column_name_cleaning(cpi, 'Geolocation')

def region_extraction(df, column_name):
    df = df.copy()
    
    REGIONS = df[column_name].unique()
    
    return REGIONS.tolist()

def commodity_extraction(df, column_name, name):
    df = df.copy()
    
    COMMODITIES = df[column_name].unique()
    
    return [f'{i} - {name}' for i in COMMODITIES.tolist()]

In [277]:
REGIONS = sorted(list(set(
    region_extraction(farmgate, 'Geolocation') +
    region_extraction(retail, 'Geolocation') +
    region_extraction(wholesale, 'Geolocation') +
    region_extraction(production, 'Geolocation') +
    region_extraction(harvested, 'Geolocation') +
    region_extraction(cpi, 'Geolocation')
)))

COMMODITIES = sorted(list(set(
    commodity_extraction(farmgate, 'Commodity', 'farmgate') +
    commodity_extraction(retail, 'Commodity', 'retail') +
    commodity_extraction(wholesale, 'Commodity', 'wholesale') +
    commodity_extraction(cpi, 'Commodity Description', 'cpi') +
    commodity_extraction(production, 'Ecosystem/Croptype', 'production') +
    commodity_extraction(harvested, 'Ecosystem/Croptype', 'harvested') 
)))

REGIONS = {
    "autonomous region in muslim mindanao (armm)" : "bangsamoro autonomous region in muslim mindanao (barmm)",
    "bangsamoro autonomous region in muslim mindanao (barmm)" : "bangsamoro autonomous region in muslim mindanao (barmm)",
    "cordillera administrative region (car)" : "cordillera administrative region (car)",
    "mimaropa region" : "mimaropa region",
    "region i (ilocos region)" : "region i (ilocos region)",
    "region ii (cagayan valley)" : "region ii (cagayan valley)",
    "region iii (central luzon)" : "region iii (central luzon)",
    "region iv-a (calabarzon)" : "region iv-a (calabarzon)",
    "region ix (zamboanga peninsula)" : "region ix (zamboanga peninsula)",
    "region v (bicol region)" : "region v (bicol region)",
    "region vi (western visayas)" : "region vi (western visayas)",
    "region v (bicol region)" : "region vi (western visayas)",
    "region vii (central visayas)" : "region vii (central visayas)",
    "region viii (eastern visayas)" : "region viii (eastern visayas)",
    "region ix (zamboanga peninsula)" : "region x (northern mindanao)",
    "region x (northern mindanao)" : "region x (northern mindanao)",
    "region xi (davao region)" : "region xi (davao region)",
    "region xii (soccsksargen)" : "region xii (soccsksargen)",
    "region xiii (caraga)" : "region xiii (caraga)"
}

In [278]:
def region_mapping(df, column_name, region_mapping):
    df = df.copy()
    
    df[column_name] = df[column_name].map(region_mapping)
    
    return df

farmgate = region_mapping(farmgate, 'Geolocation', REGIONS)
retail = region_mapping(retail, 'Geolocation', REGIONS)
wholesale = region_mapping(wholesale, 'Geolocation', REGIONS)
production = region_mapping(production, 'Geolocation', REGIONS)
harvested = region_mapping(harvested, 'Geolocation', REGIONS)
cpi = region_mapping(cpi, 'Geolocation', REGIONS)

def region_commodity_mapping(regions, commodities):
    result = []

    for region in regions:
        for commodity in commodities:
            value = f'"{region} - {commodity}",'
            result.append(value.strip().lower())
    return sorted(result)


for i in region_commodity_mapping(REGIONS, COMMODITIES):
    print(i)

"autonomous region in muslim mindanao (armm) - 01.1.1.12 - rice - cpi",
"autonomous region in muslim mindanao (armm) - 01.1.1.16 - corn - cpi",
"autonomous region in muslim mindanao (armm) - corn - harvested",
"autonomous region in muslim mindanao (armm) - corn - production",
"autonomous region in muslim mindanao (armm) - corn grits, corn native, 1 kg - retail",
"autonomous region in muslim mindanao (armm) - corn grits, loose, 1 kg - retail",
"autonomous region in muslim mindanao (armm) - corn grits, white, #10, 1 kg - retail",
"autonomous region in muslim mindanao (armm) - corn grits, white, #12, 1 kg - retail",
"autonomous region in muslim mindanao (armm) - corn grits, white, 1 kg - retail",
"autonomous region in muslim mindanao (armm) - corn grits, yellow, 1 kg - retail",
"autonomous region in muslim mindanao (armm) - corn, cracked, 1 kg - retail",
"autonomous region in muslim mindanao (armm) - corngrain [maize] white, matured - farmgate",
"autonomous region in muslim mindanao (armm

In [279]:
HEADERS = [
    "date",
    "autonomous region in muslim mindanao (armm) - 01.1.1.12 - Rice - cpi",
    "autonomous region in muslim mindanao (armm) - 01.1.1.16 - Corn - cpi",
    "autonomous region in muslim mindanao (armm) - corn - harvested",
    "autonomous region in muslim mindanao (armm) - corn - production",
    "autonomous region in muslim mindanao (armm) - corn grits, corn native, 1 kg - retail",
    "autonomous region in muslim mindanao (armm) - corn grits, loose, 1 kg - retail",
    "autonomous region in muslim mindanao (armm) - corn grits, white, #10, 1 kg - retail",
    "autonomous region in muslim mindanao (armm) - corn grits, white, #12, 1 kg - retail",
    "autonomous region in muslim mindanao (armm) - corn grits, white, 1 kg - retail",
    "autonomous region in muslim mindanao (armm) - corn grits, yellow, 1 kg - retail",
    "autonomous region in muslim mindanao (armm) - corn, cracked, 1 kg - retail",
    "autonomous region in muslim mindanao (armm) - corngrain [maize] white, matured - farmgate",
    "autonomous region in muslim mindanao (armm) - corngrain [maize] yellow, matured - farmgate",
    "autonomous region in muslim mindanao (armm) - corngrain white - wholesale",
    "autonomous region in muslim mindanao (armm) - corngrain yellow - wholesale",
    "autonomous region in muslim mindanao (armm) - corngrits white - wholesale",
    "autonomous region in muslim mindanao (armm) - corngrits yellow - wholesale",
    "autonomous region in muslim mindanao (armm) - green corn (maize, green), white - farmgate",
    "autonomous region in muslim mindanao (armm) - green corn (maize, green), yellow - farmgate",
    "autonomous region in muslim mindanao (armm) - irrigated palay - harvested",
    "autonomous region in muslim mindanao (armm) - irrigated palay - production",
    "autonomous region in muslim mindanao (armm) - palay - harvested",
    "autonomous region in muslim mindanao (armm) - palay - production",
    "autonomous region in muslim mindanao (armm) - palay [paddy] fancy, dry (conv. to 14% mc) - farmgate",
    "autonomous region in muslim mindanao (armm) - palay [paddy] other variety, dry (conv. to 14% mc) - farmgate",
    "autonomous region in muslim mindanao (armm) - rainfed palay - harvested",
    "autonomous region in muslim mindanao (armm) - rainfed palay - production",
    "autonomous region in muslim mindanao (armm) - regular milled rice (rmr) - wholesale",
    "autonomous region in muslim mindanao (armm) - rice premium - wholesale",
    "autonomous region in muslim mindanao (armm) - rice special - wholesale",
    "autonomous region in muslim mindanao (armm) - rice, regular-milled, 1 kg - retail",
    "autonomous region in muslim mindanao (armm) - rice, special, 1 kg - retail",
    "autonomous region in muslim mindanao (armm) - rice, well-milled, 1 kg - retail",
    "autonomous region in muslim mindanao (armm) - well milled rice (wmr) - wholesale",
    "autonomous region in muslim mindanao (armm) - white corn - harvested",
    "autonomous region in muslim mindanao (armm) - white corn - production",
    "autonomous region in muslim mindanao (armm) - whole corn grain, yellow, 1 kg - retail",
    "autonomous region in muslim mindanao (armm) - whole corn on the cob, native, 1 kg - retail",
    "autonomous region in muslim mindanao (armm) - whole corn on the cob, sweet corn, 1 kg - retail",
    "autonomous region in muslim mindanao (armm) - whole corn on the cob, white, 1 kg - retail",
    "autonomous region in muslim mindanao (armm) - whole corn on the cob, yellow, 1 kg - retail",
    "autonomous region in muslim mindanao (armm) - yellow corn - harvested",
    "autonomous region in muslim mindanao (armm) - yellow corn - production",
    "bangsamoro autonomous region in muslim mindanao (barmm) - 01.1.1.12 - Rice - cpi",
    "bangsamoro autonomous region in muslim mindanao (barmm) - 01.1.1.16 - Corn - cpi",
    "bangsamoro autonomous region in muslim mindanao (barmm) - corn - harvested",
    "bangsamoro autonomous region in muslim mindanao (barmm) - corn - production",
    "bangsamoro autonomous region in muslim mindanao (barmm) - corn grits, corn native, 1 kg - retail",
    "bangsamoro autonomous region in muslim mindanao (barmm) - corn grits, loose, 1 kg - retail",
    "bangsamoro autonomous region in muslim mindanao (barmm) - corn grits, white, #10, 1 kg - retail",
    "bangsamoro autonomous region in muslim mindanao (barmm) - corn grits, white, #12, 1 kg - retail",
    "bangsamoro autonomous region in muslim mindanao (barmm) - corn grits, white, 1 kg - retail",
    "bangsamoro autonomous region in muslim mindanao (barmm) - corn grits, yellow, 1 kg - retail",
    "bangsamoro autonomous region in muslim mindanao (barmm) - corn, cracked, 1 kg - retail",
    "bangsamoro autonomous region in muslim mindanao (barmm) - corngrain [maize] white, matured - farmgate",
    "bangsamoro autonomous region in muslim mindanao (barmm) - corngrain [maize] yellow, matured - farmgate",
    "bangsamoro autonomous region in muslim mindanao (barmm) - corngrain white - wholesale",
    "bangsamoro autonomous region in muslim mindanao (barmm) - corngrain yellow - wholesale",
    "bangsamoro autonomous region in muslim mindanao (barmm) - corngrits white - wholesale",
    "bangsamoro autonomous region in muslim mindanao (barmm) - corngrits yellow - wholesale",
    "bangsamoro autonomous region in muslim mindanao (barmm) - green corn (maize, green), white - farmgate",
    "bangsamoro autonomous region in muslim mindanao (barmm) - green corn (maize, green), yellow - farmgate",
    "bangsamoro autonomous region in muslim mindanao (barmm) - irrigated palay - harvested",
    "bangsamoro autonomous region in muslim mindanao (barmm) - irrigated palay - production",
    "bangsamoro autonomous region in muslim mindanao (barmm) - palay - harvested",
    "bangsamoro autonomous region in muslim mindanao (barmm) - palay - production",
    "bangsamoro autonomous region in muslim mindanao (barmm) - palay [paddy] fancy, dry (conv. to 14% mc) - farmgate",
    "bangsamoro autonomous region in muslim mindanao (barmm) - palay [paddy] other variety, dry (conv. to 14% mc) - farmgate",
    "bangsamoro autonomous region in muslim mindanao (barmm) - rainfed palay - harvested",
    "bangsamoro autonomous region in muslim mindanao (barmm) - rainfed palay - production",
    "bangsamoro autonomous region in muslim mindanao (barmm) - regular milled rice (rmr) - wholesale",
    "bangsamoro autonomous region in muslim mindanao (barmm) - rice premium - wholesale",
    "bangsamoro autonomous region in muslim mindanao (barmm) - rice special - wholesale",
    "bangsamoro autonomous region in muslim mindanao (barmm) - rice, regular-milled, 1 kg - retail",
    "bangsamoro autonomous region in muslim mindanao (barmm) - rice, special, 1 kg - retail",
    "bangsamoro autonomous region in muslim mindanao (barmm) - rice, well-milled, 1 kg - retail",
    "bangsamoro autonomous region in muslim mindanao (barmm) - well milled rice (wmr) - wholesale",
    "bangsamoro autonomous region in muslim mindanao (barmm) - white corn - harvested",
    "bangsamoro autonomous region in muslim mindanao (barmm) - white corn - production",
    "bangsamoro autonomous region in muslim mindanao (barmm) - whole corn grain, yellow, 1 kg - retail",
    "bangsamoro autonomous region in muslim mindanao (barmm) - whole corn on the cob, native, 1 kg - retail",
    "bangsamoro autonomous region in muslim mindanao (barmm) - whole corn on the cob, sweet corn, 1 kg - retail",
    "bangsamoro autonomous region in muslim mindanao (barmm) - whole corn on the cob, white, 1 kg - retail",
    "bangsamoro autonomous region in muslim mindanao (barmm) - whole corn on the cob, yellow, 1 kg - retail",
    "bangsamoro autonomous region in muslim mindanao (barmm) - yellow corn - harvested",
    "bangsamoro autonomous region in muslim mindanao (barmm) - yellow corn - production",
    "cordillera administrative region (car) - 01.1.1.12 - Rice - cpi",
    "cordillera administrative region (car) - 01.1.1.16 - Corn - cpi",
    "cordillera administrative region (car) - corn - harvested",
    "cordillera administrative region (car) - corn - production",
    "cordillera administrative region (car) - corn grits, corn native, 1 kg - retail",
    "cordillera administrative region (car) - corn grits, loose, 1 kg - retail",
    "cordillera administrative region (car) - corn grits, white, #10, 1 kg - retail",
    "cordillera administrative region (car) - corn grits, white, #12, 1 kg - retail",
    "cordillera administrative region (car) - corn grits, white, 1 kg - retail",
    "cordillera administrative region (car) - corn grits, yellow, 1 kg - retail",
    "cordillera administrative region (car) - corn, cracked, 1 kg - retail",
    "cordillera administrative region (car) - corngrain [maize] white, matured - farmgate",
    "cordillera administrative region (car) - corngrain [maize] yellow, matured - farmgate",
    "cordillera administrative region (car) - corngrain white - wholesale",
    "cordillera administrative region (car) - corngrain yellow - wholesale",
    "cordillera administrative region (car) - corngrits white - wholesale",
    "cordillera administrative region (car) - corngrits yellow - wholesale",
    "cordillera administrative region (car) - green corn (maize, green), white - farmgate",
    "cordillera administrative region (car) - green corn (maize, green), yellow - farmgate",
    "cordillera administrative region (car) - irrigated palay - harvested",
    "cordillera administrative region (car) - irrigated palay - production",
    "cordillera administrative region (car) - palay - harvested",
    "cordillera administrative region (car) - palay - production",
    "cordillera administrative region (car) - palay [paddy] fancy, dry (conv. to 14% mc) - farmgate",
    "cordillera administrative region (car) - palay [paddy] other variety, dry (conv. to 14% mc) - farmgate",
    "cordillera administrative region (car) - rainfed palay - harvested",
    "cordillera administrative region (car) - rainfed palay - production",
    "cordillera administrative region (car) - regular milled rice (rmr) - wholesale",
    "cordillera administrative region (car) - rice premium - wholesale",
    "cordillera administrative region (car) - rice special - wholesale",
    "cordillera administrative region (car) - rice, regular-milled, 1 kg - retail",
    "cordillera administrative region (car) - rice, special, 1 kg - retail",
    "cordillera administrative region (car) - rice, well-milled, 1 kg - retail",
    "cordillera administrative region (car) - well milled rice (wmr) - wholesale",
    "cordillera administrative region (car) - white corn - harvested",
    "cordillera administrative region (car) - white corn - production",
    "cordillera administrative region (car) - whole corn grain, yellow, 1 kg - retail",
    "cordillera administrative region (car) - whole corn on the cob, native, 1 kg - retail",
    "cordillera administrative region (car) - whole corn on the cob, sweet corn, 1 kg - retail",
    "cordillera administrative region (car) - whole corn on the cob, white, 1 kg - retail",
    "cordillera administrative region (car) - whole corn on the cob, yellow, 1 kg - retail",
    "cordillera administrative region (car) - yellow corn - harvested",
    "cordillera administrative region (car) - yellow corn - production",
    "mimaropa region - 01.1.1.12 - Rice - cpi",
    "mimaropa region - 01.1.1.16 - Corn - cpi",
    "mimaropa region - corn - harvested",
    "mimaropa region - corn - production",
    "mimaropa region - corn grits, corn native, 1 kg - retail",
    "mimaropa region - corn grits, loose, 1 kg - retail",
    "mimaropa region - corn grits, white, #10, 1 kg - retail",
    "mimaropa region - corn grits, white, #12, 1 kg - retail",
    "mimaropa region - corn grits, white, 1 kg - retail",
    "mimaropa region - corn grits, yellow, 1 kg - retail",
    "mimaropa region - corn, cracked, 1 kg - retail",
    "mimaropa region - corngrain [maize] white, matured - farmgate",
    "mimaropa region - corngrain [maize] yellow, matured - farmgate",
    "mimaropa region - corngrain white - wholesale",
    "mimaropa region - corngrain yellow - wholesale",
    "mimaropa region - corngrits white - wholesale",
    "mimaropa region - corngrits yellow - wholesale",
    "mimaropa region - green corn (maize, green), white - farmgate",
    "mimaropa region - green corn (maize, green), yellow - farmgate",
    "mimaropa region - irrigated palay - harvested",
    "mimaropa region - irrigated palay - production",
    "mimaropa region - palay - harvested",
    "mimaropa region - palay - production",
    "mimaropa region - palay [paddy] fancy, dry (conv. to 14% mc) - farmgate",
    "mimaropa region - palay [paddy] other variety, dry (conv. to 14% mc) - farmgate",
    "mimaropa region - rainfed palay - harvested",
    "mimaropa region - rainfed palay - production",
    "mimaropa region - regular milled rice (rmr) - wholesale",
    "mimaropa region - rice premium - wholesale",
    "mimaropa region - rice special - wholesale",
    "mimaropa region - rice, regular-milled, 1 kg - retail",
    "mimaropa region - rice, special, 1 kg - retail",
    "mimaropa region - rice, well-milled, 1 kg - retail",
    "mimaropa region - well milled rice (wmr) - wholesale",
    "mimaropa region - white corn - harvested",
    "mimaropa region - white corn - production",
    "mimaropa region - whole corn grain, yellow, 1 kg - retail",
    "mimaropa region - whole corn on the cob, native, 1 kg - retail",
    "mimaropa region - whole corn on the cob, sweet corn, 1 kg - retail",
    "mimaropa region - whole corn on the cob, white, 1 kg - retail",
    "mimaropa region - whole corn on the cob, yellow, 1 kg - retail",
    "mimaropa region - yellow corn - harvested",
    "mimaropa region - yellow corn - production",
    "region i (ilocos region) - 01.1.1.12 - Rice - cpi",
    "region i (ilocos region) - 01.1.1.16 - Corn - cpi",
    "region i (ilocos region) - corn - harvested",
    "region i (ilocos region) - corn - production",
    "region i (ilocos region) - corn grits, corn native, 1 kg - retail",
    "region i (ilocos region) - corn grits, loose, 1 kg - retail",
    "region i (ilocos region) - corn grits, white, #10, 1 kg - retail",
    "region i (ilocos region) - corn grits, white, #12, 1 kg - retail",
    "region i (ilocos region) - corn grits, white, 1 kg - retail",
    "region i (ilocos region) - corn grits, yellow, 1 kg - retail",
    "region i (ilocos region) - corn, cracked, 1 kg - retail",
    "region i (ilocos region) - corngrain [maize] white, matured - farmgate",
    "region i (ilocos region) - corngrain [maize] yellow, matured - farmgate",
    "region i (ilocos region) - corngrain white - wholesale",
    "region i (ilocos region) - corngrain yellow - wholesale",
    "region i (ilocos region) - corngrits white - wholesale",
    "region i (ilocos region) - corngrits yellow - wholesale",
    "region i (ilocos region) - green corn (maize, green), white - farmgate",
    "region i (ilocos region) - green corn (maize, green), yellow - farmgate",
    "region i (ilocos region) - irrigated palay - harvested",
    "region i (ilocos region) - irrigated palay - production",
    "region i (ilocos region) - palay - harvested",
    "region i (ilocos region) - palay - production",
    "region i (ilocos region) - palay [paddy] fancy, dry (conv. to 14% mc) - farmgate",
    "region i (ilocos region) - palay [paddy] other variety, dry (conv. to 14% mc) - farmgate",
    "region i (ilocos region) - rainfed palay - harvested",
    "region i (ilocos region) - rainfed palay - production",
    "region i (ilocos region) - regular milled rice (rmr) - wholesale",
    "region i (ilocos region) - rice premium - wholesale",
    "region i (ilocos region) - rice special - wholesale",
    "region i (ilocos region) - rice, regular-milled, 1 kg - retail",
    "region i (ilocos region) - rice, special, 1 kg - retail",
    "region i (ilocos region) - rice, well-milled, 1 kg - retail",
    "region i (ilocos region) - well milled rice (wmr) - wholesale",
    "region i (ilocos region) - white corn - harvested",
    "region i (ilocos region) - white corn - production",
    "region i (ilocos region) - whole corn grain, yellow, 1 kg - retail",
    "region i (ilocos region) - whole corn on the cob, native, 1 kg - retail",
    "region i (ilocos region) - whole corn on the cob, sweet corn, 1 kg - retail",
    "region i (ilocos region) - whole corn on the cob, white, 1 kg - retail",
    "region i (ilocos region) - whole corn on the cob, yellow, 1 kg - retail",
    "region i (ilocos region) - yellow corn - harvested",
    "region i (ilocos region) - yellow corn - production",
    "region ii (cagayan valley) - 01.1.1.12 - Rice - cpi",
    "region ii (cagayan valley) - 01.1.1.16 - Corn - cpi",
    "region ii (cagayan valley) - corn - harvested",
    "region ii (cagayan valley) - corn - production",
    "region ii (cagayan valley) - corn grits, corn native, 1 kg - retail",
    "region ii (cagayan valley) - corn grits, loose, 1 kg - retail",
    "region ii (cagayan valley) - corn grits, white, #10, 1 kg - retail",
    "region ii (cagayan valley) - corn grits, white, #12, 1 kg - retail",
    "region ii (cagayan valley) - corn grits, white, 1 kg - retail",
    "region ii (cagayan valley) - corn grits, yellow, 1 kg - retail",
    "region ii (cagayan valley) - corn, cracked, 1 kg - retail",
    "region ii (cagayan valley) - corngrain [maize] white, matured - farmgate",
    "region ii (cagayan valley) - corngrain [maize] yellow, matured - farmgate",
    "region ii (cagayan valley) - corngrain white - wholesale",
    "region ii (cagayan valley) - corngrain yellow - wholesale",
    "region ii (cagayan valley) - corngrits white - wholesale",
    "region ii (cagayan valley) - corngrits yellow - wholesale",
    "region ii (cagayan valley) - green corn (maize, green), white - farmgate",
    "region ii (cagayan valley) - green corn (maize, green), yellow - farmgate",
    "region ii (cagayan valley) - irrigated palay - harvested",
    "region ii (cagayan valley) - irrigated palay - production",
    "region ii (cagayan valley) - palay - harvested",
    "region ii (cagayan valley) - palay - production",
    "region ii (cagayan valley) - palay [paddy] fancy, dry (conv. to 14% mc) - farmgate",
    "region ii (cagayan valley) - palay [paddy] other variety, dry (conv. to 14% mc) - farmgate",
    "region ii (cagayan valley) - rainfed palay - harvested",
    "region ii (cagayan valley) - rainfed palay - production",
    "region ii (cagayan valley) - regular milled rice (rmr) - wholesale",
    "region ii (cagayan valley) - rice premium - wholesale",
    "region ii (cagayan valley) - rice special - wholesale",
    "region ii (cagayan valley) - rice, regular-milled, 1 kg - retail",
    "region ii (cagayan valley) - rice, special, 1 kg - retail",
    "region ii (cagayan valley) - rice, well-milled, 1 kg - retail",
    "region ii (cagayan valley) - well milled rice (wmr) - wholesale",
    "region ii (cagayan valley) - white corn - harvested",
    "region ii (cagayan valley) - white corn - production",
    "region ii (cagayan valley) - whole corn grain, yellow, 1 kg - retail",
    "region ii (cagayan valley) - whole corn on the cob, native, 1 kg - retail",
    "region ii (cagayan valley) - whole corn on the cob, sweet corn, 1 kg - retail",
    "region ii (cagayan valley) - whole corn on the cob, white, 1 kg - retail",
    "region ii (cagayan valley) - whole corn on the cob, yellow, 1 kg - retail",
    "region ii (cagayan valley) - yellow corn - harvested",
    "region ii (cagayan valley) - yellow corn - production",
    "region iii (central luzon) - 01.1.1.12 - Rice - cpi",
    "region iii (central luzon) - 01.1.1.16 - Corn - cpi",
    "region iii (central luzon) - corn - harvested",
    "region iii (central luzon) - corn - production",
    "region iii (central luzon) - corn grits, corn native, 1 kg - retail",
    "region iii (central luzon) - corn grits, loose, 1 kg - retail",
    "region iii (central luzon) - corn grits, white, #10, 1 kg - retail",
    "region iii (central luzon) - corn grits, white, #12, 1 kg - retail",
    "region iii (central luzon) - corn grits, white, 1 kg - retail",
    "region iii (central luzon) - corn grits, yellow, 1 kg - retail",
    "region iii (central luzon) - corn, cracked, 1 kg - retail",
    "region iii (central luzon) - corngrain [maize] white, matured - farmgate",
    "region iii (central luzon) - corngrain [maize] yellow, matured - farmgate",
    "region iii (central luzon) - corngrain white - wholesale",
    "region iii (central luzon) - corngrain yellow - wholesale",
    "region iii (central luzon) - corngrits white - wholesale",
    "region iii (central luzon) - corngrits yellow - wholesale",
    "region iii (central luzon) - green corn (maize, green), white - farmgate",
    "region iii (central luzon) - green corn (maize, green), yellow - farmgate",
    "region iii (central luzon) - irrigated palay - harvested",
    "region iii (central luzon) - irrigated palay - production",
    "region iii (central luzon) - palay - harvested",
    "region iii (central luzon) - palay - production",
    "region iii (central luzon) - palay [paddy] fancy, dry (conv. to 14% mc) - farmgate",
    "region iii (central luzon) - palay [paddy] other variety, dry (conv. to 14% mc) - farmgate",
    "region iii (central luzon) - rainfed palay - harvested",
    "region iii (central luzon) - rainfed palay - production",
    "region iii (central luzon) - regular milled rice (rmr) - wholesale",
    "region iii (central luzon) - rice premium - wholesale",
    "region iii (central luzon) - rice special - wholesale",
    "region iii (central luzon) - rice, regular-milled, 1 kg - retail",
    "region iii (central luzon) - rice, special, 1 kg - retail",
    "region iii (central luzon) - rice, well-milled, 1 kg - retail",
    "region iii (central luzon) - well milled rice (wmr) - wholesale",
    "region iii (central luzon) - white corn - harvested",
    "region iii (central luzon) - white corn - production",
    "region iii (central luzon) - whole corn grain, yellow, 1 kg - retail",
    "region iii (central luzon) - whole corn on the cob, native, 1 kg - retail",
    "region iii (central luzon) - whole corn on the cob, sweet corn, 1 kg - retail",
    "region iii (central luzon) - whole corn on the cob, white, 1 kg - retail",
    "region iii (central luzon) - whole corn on the cob, yellow, 1 kg - retail",
    "region iii (central luzon) - yellow corn - harvested",
    "region iii (central luzon) - yellow corn - production",
    "region iv-a (calabarzon) - 01.1.1.12 - Rice - cpi",
    "region iv-a (calabarzon) - 01.1.1.16 - Corn - cpi",
    "region iv-a (calabarzon) - corn - harvested",
    "region iv-a (calabarzon) - corn - production",
    "region iv-a (calabarzon) - corn grits, corn native, 1 kg - retail",
    "region iv-a (calabarzon) - corn grits, loose, 1 kg - retail",
    "region iv-a (calabarzon) - corn grits, white, #10, 1 kg - retail",
    "region iv-a (calabarzon) - corn grits, white, #12, 1 kg - retail",
    "region iv-a (calabarzon) - corn grits, white, 1 kg - retail",
    "region iv-a (calabarzon) - corn grits, yellow, 1 kg - retail",
    "region iv-a (calabarzon) - corn, cracked, 1 kg - retail",
    "region iv-a (calabarzon) - corngrain [maize] white, matured - farmgate",
    "region iv-a (calabarzon) - corngrain [maize] yellow, matured - farmgate",
    "region iv-a (calabarzon) - corngrain white - wholesale",
    "region iv-a (calabarzon) - corngrain yellow - wholesale",
    "region iv-a (calabarzon) - corngrits white - wholesale",
    "region iv-a (calabarzon) - corngrits yellow - wholesale",
    "region iv-a (calabarzon) - green corn (maize, green), white - farmgate",
    "region iv-a (calabarzon) - green corn (maize, green), yellow - farmgate",
    "region iv-a (calabarzon) - irrigated palay - harvested",
    "region iv-a (calabarzon) - irrigated palay - production",
    "region iv-a (calabarzon) - palay - harvested",
    "region iv-a (calabarzon) - palay - production",
    "region iv-a (calabarzon) - palay [paddy] fancy, dry (conv. to 14% mc) - farmgate",
    "region iv-a (calabarzon) - palay [paddy] other variety, dry (conv. to 14% mc) - farmgate",
    "region iv-a (calabarzon) - rainfed palay - harvested",
    "region iv-a (calabarzon) - rainfed palay - production",
    "region iv-a (calabarzon) - regular milled rice (rmr) - wholesale",
    "region iv-a (calabarzon) - rice premium - wholesale",
    "region iv-a (calabarzon) - rice special - wholesale",
    "region iv-a (calabarzon) - rice, regular-milled, 1 kg - retail",
    "region iv-a (calabarzon) - rice, special, 1 kg - retail",
    "region iv-a (calabarzon) - rice, well-milled, 1 kg - retail",
    "region iv-a (calabarzon) - well milled rice (wmr) - wholesale",
    "region iv-a (calabarzon) - white corn - harvested",
    "region iv-a (calabarzon) - white corn - production",
    "region iv-a (calabarzon) - whole corn grain, yellow, 1 kg - retail",
    "region iv-a (calabarzon) - whole corn on the cob, native, 1 kg - retail",
    "region iv-a (calabarzon) - whole corn on the cob, sweet corn, 1 kg - retail",
    "region iv-a (calabarzon) - whole corn on the cob, white, 1 kg - retail",
    "region iv-a (calabarzon) - whole corn on the cob, yellow, 1 kg - retail",
    "region iv-a (calabarzon) - yellow corn - harvested",
    "region iv-a (calabarzon) - yellow corn - production",
    "region ix (zamboanga peninsula) - 01.1.1.12 - Rice - cpi",
    "region ix (zamboanga peninsula) - 01.1.1.16 - Corn - cpi",
    "region ix (zamboanga peninsula) - corn - harvested",
    "region ix (zamboanga peninsula) - corn - production",
    "region ix (zamboanga peninsula) - corn grits, corn native, 1 kg - retail",
    "region ix (zamboanga peninsula) - corn grits, loose, 1 kg - retail",
    "region ix (zamboanga peninsula) - corn grits, white, #10, 1 kg - retail",
    "region ix (zamboanga peninsula) - corn grits, white, #12, 1 kg - retail",
    "region ix (zamboanga peninsula) - corn grits, white, 1 kg - retail",
    "region ix (zamboanga peninsula) - corn grits, yellow, 1 kg - retail",
    "region ix (zamboanga peninsula) - corn, cracked, 1 kg - retail",
    "region ix (zamboanga peninsula) - corngrain [maize] white, matured - farmgate",
    "region ix (zamboanga peninsula) - corngrain [maize] yellow, matured - farmgate",
    "region ix (zamboanga peninsula) - corngrain white - wholesale",
    "region ix (zamboanga peninsula) - corngrain yellow - wholesale",
    "region ix (zamboanga peninsula) - corngrits white - wholesale",
    "region ix (zamboanga peninsula) - corngrits yellow - wholesale",
    "region ix (zamboanga peninsula) - green corn (maize, green), white - farmgate",
    "region ix (zamboanga peninsula) - green corn (maize, green), yellow - farmgate",
    "region ix (zamboanga peninsula) - irrigated palay - harvested",
    "region ix (zamboanga peninsula) - irrigated palay - production",
    "region ix (zamboanga peninsula) - palay - harvested",
    "region ix (zamboanga peninsula) - palay - production",
    "region ix (zamboanga peninsula) - palay [paddy] fancy, dry (conv. to 14% mc) - farmgate",
    "region ix (zamboanga peninsula) - palay [paddy] other variety, dry (conv. to 14% mc) - farmgate",
    "region ix (zamboanga peninsula) - rainfed palay - harvested",
    "region ix (zamboanga peninsula) - rainfed palay - production",
    "region ix (zamboanga peninsula) - regular milled rice (rmr) - wholesale",
    "region ix (zamboanga peninsula) - rice premium - wholesale",
    "region ix (zamboanga peninsula) - rice special - wholesale",
    "region ix (zamboanga peninsula) - rice, regular-milled, 1 kg - retail",
    "region ix (zamboanga peninsula) - rice, special, 1 kg - retail",
    "region ix (zamboanga peninsula) - rice, well-milled, 1 kg - retail",
    "region ix (zamboanga peninsula) - well milled rice (wmr) - wholesale",
    "region ix (zamboanga peninsula) - white corn - harvested",
    "region ix (zamboanga peninsula) - white corn - production",
    "region ix (zamboanga peninsula) - whole corn grain, yellow, 1 kg - retail",
    "region ix (zamboanga peninsula) - whole corn on the cob, native, 1 kg - retail",
    "region ix (zamboanga peninsula) - whole corn on the cob, sweet corn, 1 kg - retail",
    "region ix (zamboanga peninsula) - whole corn on the cob, white, 1 kg - retail",
    "region ix (zamboanga peninsula) - whole corn on the cob, yellow, 1 kg - retail",
    "region ix (zamboanga peninsula) - yellow corn - harvested",
    "region ix (zamboanga peninsula) - yellow corn - production",
    "region v (bicol region) - 01.1.1.12 - Rice - cpi",
    "region v (bicol region) - 01.1.1.16 - Corn - cpi",
    "region v (bicol region) - corn - harvested",
    "region v (bicol region) - corn - production",
    "region v (bicol region) - corn grits, corn native, 1 kg - retail",
    "region v (bicol region) - corn grits, loose, 1 kg - retail",
    "region v (bicol region) - corn grits, white, #10, 1 kg - retail",
    "region v (bicol region) - corn grits, white, #12, 1 kg - retail",
    "region v (bicol region) - corn grits, white, 1 kg - retail",
    "region v (bicol region) - corn grits, yellow, 1 kg - retail",
    "region v (bicol region) - corn, cracked, 1 kg - retail",
    "region v (bicol region) - corngrain [maize] white, matured - farmgate",
    "region v (bicol region) - corngrain [maize] yellow, matured - farmgate",
    "region v (bicol region) - corngrain white - wholesale",
    "region v (bicol region) - corngrain yellow - wholesale",
    "region v (bicol region) - corngrits white - wholesale",
    "region v (bicol region) - corngrits yellow - wholesale",
    "region v (bicol region) - green corn (maize, green), white - farmgate",
    "region v (bicol region) - green corn (maize, green), yellow - farmgate",
    "region v (bicol region) - irrigated palay - harvested",
    "region v (bicol region) - irrigated palay - production",
    "region v (bicol region) - palay - harvested",
    "region v (bicol region) - palay - production",
    "region v (bicol region) - palay [paddy] fancy, dry (conv. to 14% mc) - farmgate",
    "region v (bicol region) - palay [paddy] other variety, dry (conv. to 14% mc) - farmgate",
    "region v (bicol region) - rainfed palay - harvested",
    "region v (bicol region) - rainfed palay - production",
    "region v (bicol region) - regular milled rice (rmr) - wholesale",
    "region v (bicol region) - rice premium - wholesale",
    "region v (bicol region) - rice special - wholesale",
    "region v (bicol region) - rice, regular-milled, 1 kg - retail",
    "region v (bicol region) - rice, special, 1 kg - retail",
    "region v (bicol region) - rice, well-milled, 1 kg - retail",
    "region v (bicol region) - well milled rice (wmr) - wholesale",
    "region v (bicol region) - white corn - harvested",
    "region v (bicol region) - white corn - production",
    "region v (bicol region) - whole corn grain, yellow, 1 kg - retail",
    "region v (bicol region) - whole corn on the cob, native, 1 kg - retail",
    "region v (bicol region) - whole corn on the cob, sweet corn, 1 kg - retail",
    "region v (bicol region) - whole corn on the cob, white, 1 kg - retail",
    "region v (bicol region) - whole corn on the cob, yellow, 1 kg - retail",
    "region v (bicol region) - yellow corn - harvested",
    "region v (bicol region) - yellow corn - production",
    "region vi (western visayas) - 01.1.1.12 - Rice - cpi",
    "region vi (western visayas) - 01.1.1.16 - Corn - cpi",
    "region vi (western visayas) - corn - harvested",
    "region vi (western visayas) - corn - production",
    "region vi (western visayas) - corn grits, corn native, 1 kg - retail",
    "region vi (western visayas) - corn grits, loose, 1 kg - retail",
    "region vi (western visayas) - corn grits, white, #10, 1 kg - retail",
    "region vi (western visayas) - corn grits, white, #12, 1 kg - retail",
    "region vi (western visayas) - corn grits, white, 1 kg - retail",
    "region vi (western visayas) - corn grits, yellow, 1 kg - retail",
    "region vi (western visayas) - corn, cracked, 1 kg - retail",
    "region vi (western visayas) - corngrain [maize] white, matured - farmgate",
    "region vi (western visayas) - corngrain [maize] yellow, matured - farmgate",
    "region vi (western visayas) - corngrain white - wholesale",
    "region vi (western visayas) - corngrain yellow - wholesale",
    "region vi (western visayas) - corngrits white - wholesale",
    "region vi (western visayas) - corngrits yellow - wholesale",
    "region vi (western visayas) - green corn (maize, green), white - farmgate",
    "region vi (western visayas) - green corn (maize, green), yellow - farmgate",
    "region vi (western visayas) - irrigated palay - harvested",
    "region vi (western visayas) - irrigated palay - production",
    "region vi (western visayas) - palay - harvested",
    "region vi (western visayas) - palay - production",
    "region vi (western visayas) - palay [paddy] fancy, dry (conv. to 14% mc) - farmgate",
    "region vi (western visayas) - palay [paddy] other variety, dry (conv. to 14% mc) - farmgate",
    "region vi (western visayas) - rainfed palay - harvested",
    "region vi (western visayas) - rainfed palay - production",
    "region vi (western visayas) - regular milled rice (rmr) - wholesale",
    "region vi (western visayas) - rice premium - wholesale",
    "region vi (western visayas) - rice special - wholesale",
    "region vi (western visayas) - rice, regular-milled, 1 kg - retail",
    "region vi (western visayas) - rice, special, 1 kg - retail",
    "region vi (western visayas) - rice, well-milled, 1 kg - retail",
    "region vi (western visayas) - well milled rice (wmr) - wholesale",
    "region vi (western visayas) - white corn - harvested",
    "region vi (western visayas) - white corn - production",
    "region vi (western visayas) - whole corn grain, yellow, 1 kg - retail",
    "region vi (western visayas) - whole corn on the cob, native, 1 kg - retail",
    "region vi (western visayas) - whole corn on the cob, sweet corn, 1 kg - retail",
    "region vi (western visayas) - whole corn on the cob, white, 1 kg - retail",
    "region vi (western visayas) - whole corn on the cob, yellow, 1 kg - retail",
    "region vi (western visayas) - yellow corn - harvested",
    "region vi (western visayas) - yellow corn - production",
    "region vii (central visayas) - 01.1.1.12 - Rice - cpi",
    "region vii (central visayas) - 01.1.1.16 - Corn - cpi",
    "region vii (central visayas) - corn - harvested",
    "region vii (central visayas) - corn - production",
    "region vii (central visayas) - corn grits, corn native, 1 kg - retail",
    "region vii (central visayas) - corn grits, loose, 1 kg - retail",
    "region vii (central visayas) - corn grits, white, #10, 1 kg - retail",
    "region vii (central visayas) - corn grits, white, #12, 1 kg - retail",
    "region vii (central visayas) - corn grits, white, 1 kg - retail",
    "region vii (central visayas) - corn grits, yellow, 1 kg - retail",
    "region vii (central visayas) - corn, cracked, 1 kg - retail",
    "region vii (central visayas) - corngrain [maize] white, matured - farmgate",
    "region vii (central visayas) - corngrain [maize] yellow, matured - farmgate",
    "region vii (central visayas) - corngrain white - wholesale",
    "region vii (central visayas) - corngrain yellow - wholesale",
    "region vii (central visayas) - corngrits white - wholesale",
    "region vii (central visayas) - corngrits yellow - wholesale",
    "region vii (central visayas) - green corn (maize, green), white - farmgate",
    "region vii (central visayas) - green corn (maize, green), yellow - farmgate",
    "region vii (central visayas) - irrigated palay - harvested",
    "region vii (central visayas) - irrigated palay - production",
    "region vii (central visayas) - palay - harvested",
    "region vii (central visayas) - palay - production",
    "region vii (central visayas) - palay [paddy] fancy, dry (conv. to 14% mc) - farmgate",
    "region vii (central visayas) - palay [paddy] other variety, dry (conv. to 14% mc) - farmgate",
    "region vii (central visayas) - rainfed palay - harvested",
    "region vii (central visayas) - rainfed palay - production",
    "region vii (central visayas) - regular milled rice (rmr) - wholesale",
    "region vii (central visayas) - rice premium - wholesale",
    "region vii (central visayas) - rice special - wholesale",
    "region vii (central visayas) - rice, regular-milled, 1 kg - retail",
    "region vii (central visayas) - rice, special, 1 kg - retail",
    "region vii (central visayas) - rice, well-milled, 1 kg - retail",
    "region vii (central visayas) - well milled rice (wmr) - wholesale",
    "region vii (central visayas) - white corn - harvested",
    "region vii (central visayas) - white corn - production",
    "region vii (central visayas) - whole corn grain, yellow, 1 kg - retail",
    "region vii (central visayas) - whole corn on the cob, native, 1 kg - retail",
    "region vii (central visayas) - whole corn on the cob, sweet corn, 1 kg - retail",
    "region vii (central visayas) - whole corn on the cob, white, 1 kg - retail",
    "region vii (central visayas) - whole corn on the cob, yellow, 1 kg - retail",
    "region vii (central visayas) - yellow corn - harvested",
    "region vii (central visayas) - yellow corn - production",
    "region viii (eastern visayas) - 01.1.1.12 - Rice - cpi",
    "region viii (eastern visayas) - 01.1.1.16 - Corn - cpi",
    "region viii (eastern visayas) - corn - harvested",
    "region viii (eastern visayas) - corn - production",
    "region viii (eastern visayas) - corn grits, corn native, 1 kg - retail",
    "region viii (eastern visayas) - corn grits, loose, 1 kg - retail",
    "region viii (eastern visayas) - corn grits, white, #10, 1 kg - retail",
    "region viii (eastern visayas) - corn grits, white, #12, 1 kg - retail",
    "region viii (eastern visayas) - corn grits, white, 1 kg - retail",
    "region viii (eastern visayas) - corn grits, yellow, 1 kg - retail",
    "region viii (eastern visayas) - corn, cracked, 1 kg - retail",
    "region viii (eastern visayas) - corngrain [maize] white, matured - farmgate",
    "region viii (eastern visayas) - corngrain [maize] yellow, matured - farmgate",
    "region viii (eastern visayas) - corngrain white - wholesale",
    "region viii (eastern visayas) - corngrain yellow - wholesale",
    "region viii (eastern visayas) - corngrits white - wholesale",
    "region viii (eastern visayas) - corngrits yellow - wholesale",
    "region viii (eastern visayas) - green corn (maize, green), white - farmgate",
    "region viii (eastern visayas) - green corn (maize, green), yellow - farmgate",
    "region viii (eastern visayas) - irrigated palay - harvested",
    "region viii (eastern visayas) - irrigated palay - production",
    "region viii (eastern visayas) - palay - harvested",
    "region viii (eastern visayas) - palay - production",
    "region viii (eastern visayas) - palay [paddy] fancy, dry (conv. to 14% mc) - farmgate",
    "region viii (eastern visayas) - palay [paddy] other variety, dry (conv. to 14% mc) - farmgate",
    "region viii (eastern visayas) - rainfed palay - harvested",
    "region viii (eastern visayas) - rainfed palay - production",
    "region viii (eastern visayas) - regular milled rice (rmr) - wholesale",
    "region viii (eastern visayas) - rice premium - wholesale",
    "region viii (eastern visayas) - rice special - wholesale",
    "region viii (eastern visayas) - rice, regular-milled, 1 kg - retail",
    "region viii (eastern visayas) - rice, special, 1 kg - retail",
    "region viii (eastern visayas) - rice, well-milled, 1 kg - retail",
    "region viii (eastern visayas) - well milled rice (wmr) - wholesale",
    "region viii (eastern visayas) - white corn - harvested",
    "region viii (eastern visayas) - white corn - production",
    "region viii (eastern visayas) - whole corn grain, yellow, 1 kg - retail",
    "region viii (eastern visayas) - whole corn on the cob, native, 1 kg - retail",
    "region viii (eastern visayas) - whole corn on the cob, sweet corn, 1 kg - retail",
    "region viii (eastern visayas) - whole corn on the cob, white, 1 kg - retail",
    "region viii (eastern visayas) - whole corn on the cob, yellow, 1 kg - retail",
    "region viii (eastern visayas) - yellow corn - harvested",
    "region viii (eastern visayas) - yellow corn - production",
    "region x (northern mindanao) - 01.1.1.12 - Rice - cpi",
    "region x (northern mindanao) - 01.1.1.16 - Corn - cpi",
    "region x (northern mindanao) - corn - harvested",
    "region x (northern mindanao) - corn - production",
    "region x (northern mindanao) - corn grits, corn native, 1 kg - retail",
    "region x (northern mindanao) - corn grits, loose, 1 kg - retail",
    "region x (northern mindanao) - corn grits, white, #10, 1 kg - retail",
    "region x (northern mindanao) - corn grits, white, #12, 1 kg - retail",
    "region x (northern mindanao) - corn grits, white, 1 kg - retail",
    "region x (northern mindanao) - corn grits, yellow, 1 kg - retail",
    "region x (northern mindanao) - corn, cracked, 1 kg - retail",
    "region x (northern mindanao) - corngrain [maize] white, matured - farmgate",
    "region x (northern mindanao) - corngrain [maize] yellow, matured - farmgate",
    "region x (northern mindanao) - corngrain white - wholesale",
    "region x (northern mindanao) - corngrain yellow - wholesale",
    "region x (northern mindanao) - corngrits white - wholesale",
    "region x (northern mindanao) - corngrits yellow - wholesale",
    "region x (northern mindanao) - green corn (maize, green), white - farmgate",
    "region x (northern mindanao) - green corn (maize, green), yellow - farmgate",
    "region x (northern mindanao) - irrigated palay - harvested",
    "region x (northern mindanao) - irrigated palay - production",
    "region x (northern mindanao) - palay - harvested",
    "region x (northern mindanao) - palay - production",
    "region x (northern mindanao) - palay [paddy] fancy, dry (conv. to 14% mc) - farmgate",
    "region x (northern mindanao) - palay [paddy] other variety, dry (conv. to 14% mc) - farmgate",
    "region x (northern mindanao) - rainfed palay - harvested",
    "region x (northern mindanao) - rainfed palay - production",
    "region x (northern mindanao) - regular milled rice (rmr) - wholesale",
    "region x (northern mindanao) - rice premium - wholesale",
    "region x (northern mindanao) - rice special - wholesale",
    "region x (northern mindanao) - rice, regular-milled, 1 kg - retail",
    "region x (northern mindanao) - rice, special, 1 kg - retail",
    "region x (northern mindanao) - rice, well-milled, 1 kg - retail",
    "region x (northern mindanao) - well milled rice (wmr) - wholesale",
    "region x (northern mindanao) - white corn - harvested",
    "region x (northern mindanao) - white corn - production",
    "region x (northern mindanao) - whole corn grain, yellow, 1 kg - retail",
    "region x (northern mindanao) - whole corn on the cob, native, 1 kg - retail",
    "region x (northern mindanao) - whole corn on the cob, sweet corn, 1 kg - retail",
    "region x (northern mindanao) - whole corn on the cob, white, 1 kg - retail",
    "region x (northern mindanao) - whole corn on the cob, yellow, 1 kg - retail",
    "region x (northern mindanao) - yellow corn - harvested",
    "region x (northern mindanao) - yellow corn - production",
    "region xi (davao region) - 01.1.1.12 - Rice - cpi",
    "region xi (davao region) - 01.1.1.16 - Corn - cpi",
    "region xi (davao region) - corn - harvested",
    "region xi (davao region) - corn - production",
    "region xi (davao region) - corn grits, corn native, 1 kg - retail",
    "region xi (davao region) - corn grits, loose, 1 kg - retail",
    "region xi (davao region) - corn grits, white, #10, 1 kg - retail",
    "region xi (davao region) - corn grits, white, #12, 1 kg - retail",
    "region xi (davao region) - corn grits, white, 1 kg - retail",
    "region xi (davao region) - corn grits, yellow, 1 kg - retail",
    "region xi (davao region) - corn, cracked, 1 kg - retail",
    "region xi (davao region) - corngrain [maize] white, matured - farmgate",
    "region xi (davao region) - corngrain [maize] yellow, matured - farmgate",
    "region xi (davao region) - corngrain white - wholesale",
    "region xi (davao region) - corngrain yellow - wholesale",
    "region xi (davao region) - corngrits white - wholesale",
    "region xi (davao region) - corngrits yellow - wholesale",
    "region xi (davao region) - green corn (maize, green), white - farmgate",
    "region xi (davao region) - green corn (maize, green), yellow - farmgate",
    "region xi (davao region) - irrigated palay - harvested",
    "region xi (davao region) - irrigated palay - production",
    "region xi (davao region) - palay - harvested",
    "region xi (davao region) - palay - production",
    "region xi (davao region) - palay [paddy] fancy, dry (conv. to 14% mc) - farmgate",
    "region xi (davao region) - palay [paddy] other variety, dry (conv. to 14% mc) - farmgate",
    "region xi (davao region) - rainfed palay - harvested",
    "region xi (davao region) - rainfed palay - production",
    "region xi (davao region) - regular milled rice (rmr) - wholesale",
    "region xi (davao region) - rice premium - wholesale",
    "region xi (davao region) - rice special - wholesale",
    "region xi (davao region) - rice, regular-milled, 1 kg - retail",
    "region xi (davao region) - rice, special, 1 kg - retail",
    "region xi (davao region) - rice, well-milled, 1 kg - retail",
    "region xi (davao region) - well milled rice (wmr) - wholesale",
    "region xi (davao region) - white corn - harvested",
    "region xi (davao region) - white corn - production",
    "region xi (davao region) - whole corn grain, yellow, 1 kg - retail",
    "region xi (davao region) - whole corn on the cob, native, 1 kg - retail",
    "region xi (davao region) - whole corn on the cob, sweet corn, 1 kg - retail",
    "region xi (davao region) - whole corn on the cob, white, 1 kg - retail",
    "region xi (davao region) - whole corn on the cob, yellow, 1 kg - retail",
    "region xi (davao region) - yellow corn - harvested",
    "region xi (davao region) - yellow corn - production",
    "region xii (soccsksargen) - 01.1.1.12 - Rice - cpi",
    "region xii (soccsksargen) - 01.1.1.16 - Corn - cpi",
    "region xii (soccsksargen) - corn - harvested",
    "region xii (soccsksargen) - corn - production",
    "region xii (soccsksargen) - corn grits, corn native, 1 kg - retail",
    "region xii (soccsksargen) - corn grits, loose, 1 kg - retail",
    "region xii (soccsksargen) - corn grits, white, #10, 1 kg - retail",
    "region xii (soccsksargen) - corn grits, white, #12, 1 kg - retail",
    "region xii (soccsksargen) - corn grits, white, 1 kg - retail",
    "region xii (soccsksargen) - corn grits, yellow, 1 kg - retail",
    "region xii (soccsksargen) - corn, cracked, 1 kg - retail",
    "region xii (soccsksargen) - corngrain [maize] white, matured - farmgate",
    "region xii (soccsksargen) - corngrain [maize] yellow, matured - farmgate",
    "region xii (soccsksargen) - corngrain white - wholesale",
    "region xii (soccsksargen) - corngrain yellow - wholesale",
    "region xii (soccsksargen) - corngrits white - wholesale",
    "region xii (soccsksargen) - corngrits yellow - wholesale",
    "region xii (soccsksargen) - green corn (maize, green), white - farmgate",
    "region xii (soccsksargen) - green corn (maize, green), yellow - farmgate",
    "region xii (soccsksargen) - irrigated palay - harvested",
    "region xii (soccsksargen) - irrigated palay - production",
    "region xii (soccsksargen) - palay - harvested",
    "region xii (soccsksargen) - palay - production",
    "region xii (soccsksargen) - palay [paddy] fancy, dry (conv. to 14% mc) - farmgate",
    "region xii (soccsksargen) - palay [paddy] other variety, dry (conv. to 14% mc) - farmgate",
    "region xii (soccsksargen) - rainfed palay - harvested",
    "region xii (soccsksargen) - rainfed palay - production",
    "region xii (soccsksargen) - regular milled rice (rmr) - wholesale",
    "region xii (soccsksargen) - rice premium - wholesale",
    "region xii (soccsksargen) - rice special - wholesale",
    "region xii (soccsksargen) - rice, regular-milled, 1 kg - retail",
    "region xii (soccsksargen) - rice, special, 1 kg - retail",
    "region xii (soccsksargen) - rice, well-milled, 1 kg - retail",
    "region xii (soccsksargen) - well milled rice (wmr) - wholesale",
    "region xii (soccsksargen) - white corn - harvested",
    "region xii (soccsksargen) - white corn - production",
    "region xii (soccsksargen) - whole corn grain, yellow, 1 kg - retail",
    "region xii (soccsksargen) - whole corn on the cob, native, 1 kg - retail",
    "region xii (soccsksargen) - whole corn on the cob, sweet corn, 1 kg - retail",
    "region xii (soccsksargen) - whole corn on the cob, white, 1 kg - retail",
    "region xii (soccsksargen) - whole corn on the cob, yellow, 1 kg - retail",
    "region xii (soccsksargen) - yellow corn - harvested",
    "region xii (soccsksargen) - yellow corn - production",
    "region xiii (caraga) - 01.1.1.12 - Rice - cpi",
    "region xiii (caraga) - 01.1.1.16 - Corn - cpi",
    "region xiii (caraga) - corn - harvested",
    "region xiii (caraga) - corn - production",
    "region xiii (caraga) - corn grits, corn native, 1 kg - retail",
    "region xiii (caraga) - corn grits, loose, 1 kg - retail",
    "region xiii (caraga) - corn grits, white, #10, 1 kg - retail",
    "region xiii (caraga) - corn grits, white, #12, 1 kg - retail",
    "region xiii (caraga) - corn grits, white, 1 kg - retail",
    "region xiii (caraga) - corn grits, yellow, 1 kg - retail",
    "region xiii (caraga) - corn, cracked, 1 kg - retail",
    "region xiii (caraga) - corngrain [maize] white, matured - farmgate",
    "region xiii (caraga) - corngrain [maize] yellow, matured - farmgate",
    "region xiii (caraga) - corngrain white - wholesale",
    "region xiii (caraga) - corngrain yellow - wholesale",
    "region xiii (caraga) - corngrits white - wholesale",
    "region xiii (caraga) - corngrits yellow - wholesale",
    "region xiii (caraga) - green corn (maize, green), white - farmgate",
    "region xiii (caraga) - green corn (maize, green), yellow - farmgate",
    "region xiii (caraga) - irrigated palay - harvested",
    "region xiii (caraga) - irrigated palay - production",
    "region xiii (caraga) - palay - harvested",
    "region xiii (caraga) - palay - production",
    "region xiii (caraga) - palay [paddy] fancy, dry (conv. to 14% mc) - farmgate",
    "region xiii (caraga) - palay [paddy] other variety, dry (conv. to 14% mc) - farmgate",
    "region xiii (caraga) - rainfed palay - harvested",
    "region xiii (caraga) - rainfed palay - production",
    "region xiii (caraga) - regular milled rice (rmr) - wholesale",
    "region xiii (caraga) - rice premium - wholesale",
    "region xiii (caraga) - rice special - wholesale",
    "region xiii (caraga) - rice, regular-milled, 1 kg - retail",
    "region xiii (caraga) - rice, special, 1 kg - retail",
    "region xiii (caraga) - rice, well-milled, 1 kg - retail",
    "region xiii (caraga) - well milled rice (wmr) - wholesale",
    "region xiii (caraga) - white corn - harvested",
    "region xiii (caraga) - white corn - production",
    "region xiii (caraga) - whole corn grain, yellow, 1 kg - retail",
    "region xiii (caraga) - whole corn on the cob, native, 1 kg - retail",
    "region xiii (caraga) - whole corn on the cob, sweet corn, 1 kg - retail",
    "region xiii (caraga) - whole corn on the cob, white, 1 kg - retail",
    "region xiii (caraga) - whole corn on the cob, yellow, 1 kg - retail",
    "region xiii (caraga) - yellow corn - harvested",
    "region xiii (caraga) - yellow corn - production"
]

In [ ]:
DATES = [
    '2020 January' = ['2020 January'], 
    '2020 February' = ['2020 February'],
    '2020 March' = ['2020 March'], 
    '2020 April' = ['2020 April'], 
    '2020 May' = ['2020 May'], 
    '2020 June' = ['2020 '], 
    '2020 July',
    '2020 August', 
    '2020 September', 
    '2020 October', 
    '2020 November',
    '2020 December', 
    '2021 January', 
    '2021 February', 
    '2021 March',
    '2021 April', 
    '2021 May', 
    '2021 June', 
    '2021 July', 
    '2021 August',
    '2021 September', 
    '2021 October', 
    '2021 November', 
    '2021 December',
    '2022 January', 
    '2022 February', 
    '2022 March', 
    '2022 April', 
    '2022 May',
    '2022 June', 
    '2022 July', 
    '2022 August', 
    '2022 September',
    '2022 October', 
    '2022 November', 
    '2022 December', 
    '2023 January',
    '2023 February', 
    '2023 March', 
    '2023 April', 
    '2023 May', 
    '2023 June',
    '2023 July', 
    '2023 August', 
    '2023 September', 
    '2023 October',
    '2023 November', 
    '2023 December', 
    '2024 January', 
    '2024 February',
    '2024 March', 
    '2024 April', 
    '2024 May', 
    '2024 June', 
    '2024 July',
    '2024 August', 
    '2024 September', 
    '2024 October', 
    '2024 November',
    '2024 December'
]

farmgate_copy['Date'].unique()

array(['2020 January', '2020 February', '2020 March', '2020 April',
       '2020 May', '2020 June', '2020 July', '2020 August',
       '2020 September', '2020 October', '2020 November', '2020 December',
       '2021 January', '2021 February', '2021 March', '2021 April',
       '2021 May', '2021 June', '2021 July', '2021 August',
       '2021 September', '2021 October', '2021 November', '2021 December',
       '2022 January', '2022 February', '2022 March', '2022 April',
       '2022 May', '2022 June', '2022 July', '2022 August',
       '2022 September', '2022 October', '2022 November', '2022 December',
       '2023 January', '2023 February', '2023 March', '2023 April',
       '2023 May', '2023 June', '2023 July', '2023 August',
       '2023 September', '2023 October', '2023 November', '2023 December',
       '2024 January', '2024 February', '2024 March', '2024 April',
       '2024 May', '2024 June', '2024 July', '2024 August',
       '2024 September', '2024 October', '2024 November', '2